## Task 1: Preprocess dataset

In [ ]:
from datasets import load_dataset
from matplotlib import pyplot as plt

# Load GoEmotions dataset
dataset = load_dataset("google-research-datasets/go_emotions")

print(dataset['train'])

Dataset({
    features: ['text', 'labels', 'id'],
    num_rows: 43410
})


In [ ]:
def filter_single_label(example):
    return len(example['labels']) == 1

# Apply filtering
train_data = dataset['train'].filter(filter_single_label)
val_data = dataset['validation'].filter(filter_single_label)
test_data = dataset['test'].filter(filter_single_label)


For Baseline Models (SR, RF, XGBoost):Use TfidfVectorizer from scikit-learn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_data['text'])
X_val = vectorizer.transform(val_data['text'])
X_test = vectorizer.transform(test_data['text'])

y_train = train_data['labels']
y_val = val_data['labels']
y_test = test_data['labels']


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Example: Random Forest
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Validate and Test
y_pred = clf.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))


/home/mcb/users/bwang6/miniconda3/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Test Accuracy: 0.5514161220043573


For Naive Bayes: Use CountVectorizer for bag-of-words representation

In [ ]:
import numpy as np
import itertools
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import datasets
np.random.seed(1234)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_data['text'])
X_val = vectorizer.transform(val_data['text'])
X_test = vectorizer.transform(test_data['text'])

y_train = train_data['labels']
y_val = val_data['labels']
y_test = test_data['labels']

In [ ]:
class GaussianNaiveBayes:

    def __init__(self):
        return

    def fit(self, x, y):
        if hasattr(x, "toarray"):
            x = x.toarray()
        y = np.array(y)
        print(f"X_train.shape = {x.shape}")
        print(f"y_train.shape = {y.shape}")


        N, D = x.shape
        C = np.max(y) + 1
        # one parameter for each feature conditioned on each class
        mu, sigma = np.zeros((C,D)), np.zeros((C,D))
        Nc = np.zeros(C) # number of instances in class c
        # for each class get the MLE for the mean and std
        for c in range(C):
            x_c = x[y == c]                           #slice all the elements from class c
            print(f"Class {c}: x_c.shape = {x_c.shape}")
            Nc[c] = x_c.shape[0]                      #get number of elements of class c
            mu[c,:] = np.mean(x_c,0)                  #mean of features of class c
            sigma[c,:] = np.std(x_c, 0)               #std of features of class c

        self.mu = mu                                  # C x D
        self.sigma = sigma                            # C x D
        self.pi = (Nc+1)/(N+C)                        #Laplace smoothing (using alpha_c=1 for all c) you can derive using Dirichlet's distribution
        return self


In [ ]:
def logsumexp(Z):                                                # dimension C x N
    Zmax = np.max(Z,axis=0)[None,:]                              # max over C
    log_sum_exp = Zmax + np.log(np.sum(np.exp(Z - Zmax), axis=0))
    return log_sum_exp

def predict(self, xt):
    Nt, D = xt.shape
    # for numerical stability we work in the log domain
    # we add a dimension because this is added to the log-likelihood matrix
    # that assigns a likelihood for each class (C) to each test point, and so it is C x N
    log_prior = np.log(self.pi)[:, None]
    # logarithm of the likelihood term for Gaussian
    # the first two terms are the logarithm of the normalization term in the Gaussian and the final term is the exponent in the Gaussian.
    # Notice that we are adding dimensions (using None) to model parameters and data to make this evaluation.
    # The reason is that sigma and mu are C x D, while the data x is N x D. We operate on a C x N x D shape by increasing the number of dimensions when needed
    log_likelihood = -.5 * np.log(2*np.pi) - np.log(self.sigma[:,None,:]) -.5 * (((xt[None,:,:] - self.mu[:,None,:])/self.sigma[:,None,:])**2)
    # now we sum over the feature dimension to get a C x N matrix (this has the log-likelihood for each class-test point combination)
    log_likelihood = np.sum(log_likelihood, axis=2)
    # posterior calculation
    log_posterior = log_prior + log_likelihood
    posterior = np.exp(log_posterior - logsumexp(log_posterior))
    return posterior.T                                                  # dimension N x C

GaussianNaiveBayes.predict = predict

In [ ]:
model = GaussianNaiveBayes()
model.fit(X_train, y_train)
y_prob = model.predict(X_test)
y_pred = np.argmax(y_prob, 1)
accuracy = np.sum(y_pred == y_test)/y_pred.shape[0]
print(f'test accuracy: {accuracy}')

x.shape = (36308, 5000)
y.shape = (36308,)
Class 0: x_c.shape = (2710, 5000)


AxisError: axis 0 is out of bounds for array of dimension 0

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))


Test Accuracy: 0.49869281045751634


/home/mcb/users/bwang6/miniconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


For LLM (HuggingFace Transformers): Use a tokenizer from the transformers library

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_data(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_encodings = train_data.map(tokenize_data, batched=True)
val_encodings = val_data.map(tokenize_data, batched=True)
test_encodings = test_data.map(tokenize_data, batched=True)

In [ ]:
print(train_encodings)

Dataset({
    features: ['text', 'labels', 'id', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36308
})


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_data['labels'])
)

val_dataset = TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_encodings['attention_mask']),
    torch.tensor(val_data['labels'])
)

test_dataset = TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_data['labels'])
)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [ ]:
from transformers import AutoTokenizer, AutoModel

# Define the path to save the model
save_path = "./pretrained_model"

# Download and save the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", cache_dir=save_path)
base_model = AutoModel.from_pretrained("bert-base-uncased", cache_dir=save_path)

# Verify the save path
print(f"Model and tokenizer saved at: {save_path}")


Model and tokenizer saved at: ./pretrained_model


In [ ]:

import torch.nn as nn
class CustomBertModel(nn.Module):
    def __init__(self, base_model, num_labels):
        super().__init__()
        self.bert = base_model
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(base_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, label=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if label is not None:
            criterion = nn.CrossEntropyLoss()
            loss = criterion(logits, label)

        return {'loss':loss, "logits":logits} if loss is not None else {'logits':logits}
    def save_pretrained(self, save_directory):
        # Save tokenizer and model to the specified directory
        torch.save(self.state_dict(), f"{save_directory}/pytorch_model.bin")

In [ ]:
num_labels =  28
model = CustomBertModel(base_model,num_labels)

In [ ]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm import tqdm

# Define hyperparameters
learning_rate = 5e-5
epochs = 3
batch_size = 8
# Move model to GPU if available
device = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
# Total training steps (for learning rate scheduler)
num_training_steps = len(train_loader) * epochs
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
# Loss function
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(epochs):

    print(f"Epoch {epoch + 1}/{epochs}")
    model.train()
    train_loss=0
    for batch in tqdm(train_loader,desc='training'):
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids,attention_mask=attention_mask)
        logits = outputs['logits']
        labels = labels.squeeze(-1)
        # compute loss
        loss = criterion(logits, labels)
        train_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Training loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validation'):
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)
            # Forward
            outputs = model(input_ids = input_ids, attention_mask = attention_mask)
            logits = outputs["logits"]
            labels = labels.squeeze(-1)
            # Compute loss
            loss = criterion(logits, labels)
            val_loss += loss.item()

            # Compute accuracy
            predictions = torch.argmax(logits, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = correct / total
    print(f"Validation loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.4f}")


Epoch 1/3


training: 100%|██████████| 2270/2270 [04:48<00:00,  7.86it/s]


Training loss: 1.5494


Validation: 100%|██████████| 285/285 [00:10<00:00, 27.52it/s]


Validation loss: 1.2559, Accuracy: 0.6223
Epoch 2/3


...

Validation loss: 1.2452, Accuracy: 0.6242
Epoch 3/3


training: 100%|██████████| 2270/2270 [04:45<00:00,  7.95it/s]


Training loss: 0.7063


Validation: 100%|██████████| 285/285 [00:10<00:00, 27.48it/s]

Validation loss: 1.3452, Accuracy: 0.6209


In [ ]:
# Save the model and tokenizer
model.save_pretrained("./custom_bert_model")
tokenizer.save_pretrained("./custom_bert_model")


('./custom_bert_model/tokenizer_config.json',
 './custom_bert_model/special_tokens_map.json',
 './custom_bert_model/vocab.txt',
 './custom_bert_model/added_tokens.json',
 './custom_bert_model/tokenizer.json')

In [ ]:
from transformers import pipeline

# Load fine-tuned model
emotion_classifier = pipeline("text-classification", model="./custom_bert_model", tokenizer="./custom_bert_model")

# Test on custom text
text = "I'm so happy to see you!"
predictions = emotion_classifier(text)
print(predictions)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_17', 'score': 0.912319004535675}]


In [ ]:
model.save_pretrained("./custom_bert_model")
tokenizer.save_pretrained("./custom_bert_model")

# Naive Bayes

## Data processing

In [ ]:
df = pd.read_parquet("hf://datasets/google-research-datasets/go_emotions/raw/train-00000-of-00001.parquet")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
labels = df.columns[9:]

In [ ]:
df1 = df[df[labels].sum(axis=1) == 1]

In [ ]:
comments = df1['text'].tolist()
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(comments)
y = df1[labels].values
y = np.argmax(y, axis=1)

# Naive Bayes Implementation

In [ ]:
# Note X must be a sparse matrix. This is because the session crashes otherwise
# y is integer encoded not one hot encoded

class NaiveBayes:
    def __init__(self):
        self.px = None
        self.py = None

    def fit(self, X, y):
        # Shape of X is (N, L) where L is the length of the embedding vectors, X is a sparse matrix
        # Shape of y is (N,) where N is sample size

        self.X = X    # Bag of words representation
        self.y = y    # Integer labeled
        n = X.shape[0]
        C = len(np.unique(y))

        # Compute class priors (py)
        for c in range(C):
            self.py = np.bincount(y)[c] / n

        # Initialize px as a list to store likelihoods
        self.px = []

        for c in range(C):
            # Select samples where the class is c
            y_c = (y == c)  # Binary mask for samples with class c
            X_c = X[y_c == 1]  # Extract samples where class is c

            # Compute the likelihood P(x_i | y_c) for each feature
            px_c = (X_c.sum(axis=0) + 1) / (y_c.sum() + X.shape[1])
            px_c = np.asarray(px_c).ravel()  # Ensure it's a dense 1D array

            self.px.append(px_c)  # Add the likelihoods for class c

        # Convert px to numpy array of shape (C, L)
        self.px = np.array(self.px)

    def predict(self, X):
        # Compute the log of the posterior probabilities for each class
        log_py = np.log(self.py)  # Log of class priors
        log_px = np.log(self.px)  # Log of feature likelihoods

        # Compute log-posterior for each class (N samples, C classes)
        log_posterior = X.dot(log_px.T) + log_py  # `X` remains sparse

        # Return the class with the highest posterior probability for each sample
        return np.argmax(log_posterior, axis=1)

    def evaluate_acc(self, Y, Yh):
        return np.mean(Y == Yh)


In [ ]:
model = NaiveBayes()
model.fit(X, y)
Yh = model.predict(X)
model.evaluate_acc(y, Yh)

0.41989873123035737

In [ ]:
class_counts = np.bincount(y)
print("Class counts:", class_counts)


Class counts: [10531  6130  5202  8342 11259  3523  4938  5885  2147  4706  7686  2914
  1433  3020  1778  7075   351  4329  4957   796  4519   690  4714   788
  1510  3827  3472 55298]


In [ ]:
baseline_accuracy = np.max(class_counts) / len(y)
print(f"Baseline Accuracy: {baseline_accuracy:.2f}")


Baseline Accuracy: 0.32
